![CMCC](http://cmcc.ufabc.edu.br/images/logo_site.jpg)
# **Spark + Python = PySpark**

#### Esse notebook introduz os conceitos básicos do Spark através de sua interface com a linguagem Python. Como aplicação inicial faremos o clássico examplo de contador de palavras . Com esse exemplo é possível entender a lógica de programação funcional para as diversas tarefas de exploração de dados distribuídos.

#### Para isso utilizaremos a base de e-mails da Hillary Clinton, fornecida pelo [Kaggle](http://kaggle.com).  Veremos que esse mesmo algoritmo pode ser empregado em textos de qualquer tamanho.

#### ** Esse notebook contém:  **
#### *Parte 1:* Criando uma base RDD e RDDs de tuplas
#### *Parte 2:* Manipulando RDDs de tuplas
#### *Parte 3:* Encontrando palavras únicas
#### *Parte 4:* Aplicar contagem de palavras em um arquivo

### ** Part 1: Criando e Manipulando RDDs **

#### Nessa parte do notebook vamos criar uma base RDD a partir de uma lista com o comando `parallelize`.

#### ** (1a) Criando uma base RDD **
#### Podemos criar uma base RDD de diversos tipos e fonte do Python com o comando `sc.parallelize(fonte, particoes)`, sendo fonte uma variável contendo os dados (ex.: uma lista) e particoes o número de partições para trabalhar em paralelo.

In [1]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print type(palavrasRDD)

<class 'pyspark.rdd.RDD'>


#### ** (1b) Plural **

#### Vamos criar uma função que transforma uma palavra no plural adicionando uma letra 's' ao final da string. Em seguida vamos utilizar a função `map()` para aplicar a transformação em cada palavra do RDD.

#### Em Python (e muitas outras linguagens) a concatenação de strings é custosa. Uma alternativa melhor é criar uma nova string utilizando [`str.format()`](https://docs.python.org/2/library/string.html#format-string-syntax).

#### Nota: a string entre os conjuntos de três aspas representa a documentação da função. Essa documentação é exibida com o comando `help()`. Vamos utilizar a padronização de documentação sugerida para o Python, manteremos essa documentação em inglês.

In [2]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return palavra + 's'

print Plural('gato')

gatos


In [3]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



#### ** (1c) Aplicando a função ao RDD **
#### Transforme cada palavra do nosso RDD em plural usando [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) 

#### Em seguida, utilizaremos o comando  [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) que retorna a RDD como uma lista do Python.

In [6]:
# EXERCICIO
pluralRDD = palavrasRDD.map(Plural)
print pluralRDD.collect()

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


#### ** Nota: ** utilize o comando `collect()` apenas quando tiver certeza de que a lista caberá na memória. Para gravar os resultados de volta em arquivo texto ou base de dados utilizaremos outro comando.

#### ** (1d) Utilizando uma função `lambda` **
#### Repita a criação de um RDD de plurais, porém utilizando uma função lambda.

In [8]:
# EXERCICIO
pluralLambdaRDD = palavrasRDD.map(lambda x: x+'s')
print pluralLambdaRDD.collect()

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


#### ** (1e) Tamanho de cada palavra **
#### Agora use `map()` e uma função `lambda` para retornar o número de caracteres em cada palavra. Utilize `collect()` para armazenar o resultado em forma de listas na variável destino.

In [12]:
# EXERCICIO
pluralTamanho = (pluralRDD
                 .map(lambda x: len(x))
                 )
print pluralTamanho.collect()

[5, 9, 5, 5, 5]


#### ** (1f) RDDs de pares e tuplas **

#### Para contar a frequência de cada palavra de maneira distribuída, primeiro devemos atribuir um valor para cada palavra do RDD. Isso irá gerar um base de dados (chave, valor). Desse modo podemos agrupar a base através da chave, calculando a soma dos valores atribuídos. No nosso caso, vamos atribuir o valor `1` para cada palavra.

#### Um RDD contendo a estrutura de tupla chave-valor `(k,v) ` é chamada de RDD de tuplas ou *pair RDD*.

#### Vamos criar nosso RDD de pares usando a transformação  `map()` com uma função `lambda()`.

In [14]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x: (x,1))
print palavraPar.collect()

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


### ** Parte 2: Manipulando RDD de tuplas **

#### Vamos manipular nossa RDD para contar as palavras do texto.

#### ** (2c) `reduceByKey` **
#### Um comando mais interessante para a contagem é o  [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) que cria uma nova RDD de tuplas.

#### Essa transformação aplica a transformação `reduce()` vista na aula anterior para os valores de cada chave. Dessa forma, a função de transformação pode ser aplicada em cada partição local para depois ser enviada para redistribuição de partições, reduzindo o total de dados sendo movidos e não mantendo listas grandes na memória.

In [15]:
# EXERCICIO
contagem = palavraPar.reduceByKey(lambda x,y: x+y)
print contagem.collect()

[('rato', 2), ('elefante', 1), ('gato', 2)]


#### ** (2d) Agrupando os comandos **

#### A forma mais usual de realizar essa tarefa, partindo do nosso RDD palavrasRDD, é encadear os comandos map e reduceByKey em uma linha de comando.

In [16]:
# EXERCICIO
contagemFinal = (palavrasRDD
                 .map(lambda x: (x,1))
                 .reduceByKey(lambda x,y: x+y)
                 .collect())
print contagemFinal

[('rato', 2), ('elefante', 1), ('gato', 2)]


### ** Parte 4: Aplicar nosso algoritmo em um arquivo **

#### ** (4b) Normalizando o texto **

#### Quando trabalhamos com dados reais, geralmente precisamos padronizar os atributos de tal forma que diferenças sutis por conta de erro de medição ou diferença de normatização, sejam desconsideradas. Para o próximo passo vamos padronizar o texto para:
  + #### Padronizar a capitalização das palavras (tudo maiúsculo ou tudo minúsculo).
  + #### Remover pontuação.
  + #### Remover espaços no início e no final da palavra.
 
#### Crie uma função `removerPontuacao` que converte todo o texto para minúscula, remove qualquer pontuação e espaços em branco no início ou final da palavra. Para isso, utilize a biblioteca [re](https://docs.python.org/2/library/re.html) para remover todo texto que não seja letra, número ou espaço, encadeando com as funções de string para remover espaços em branco e converter para minúscula (veja [Strings](https://docs.python.org/2/library/stdtypes.html?highlight=str.lower#string-methods)).

In [1]:
# EXERCICIO
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^a-zA-Z0-9 ]+','',re.sub('_',' ',texto)).strip().lower()
print removerPontuacao('Ola, quem esta ai??!')
print removerPontuacao(' Sem espaco e_sublinhado!')

Creating SparkContext as 'sc'
Creating SqlContext as 'sqlContext'
Creating HiveContext as 'hiveContext'
ola quem esta ai
sem espaco e sublinhado

#### ** (4c) Carregando arquivo texto  **

#### Para converter um texto em uma RDD, utilizamos a função `textFile()` que recebe como entrada o nome do arquivo texto que queremos utilizar e o número de partições.

#### O nome do arquivo texto pode se referir a um arquivo local ou uma URI de arquivo distribuído (ex.: hdfs://).

#### Vamos também aplicar a função `removerPontuacao()` para normalizar o texto e verificar as 15 primeiras linhas com o comando `take()`.

In [2]:
import os.path

hillaryRDD = sc.textFile('EmailsProcessed.csv')
    
header = hillaryRDD.take(1)[0]
print header.split(',')

hillaryDataRDD = (hillaryRDD
                  .filter(lambda x: x!=header)
                  .map(lambda x: x.split('~')[-1])
                  .map(removerPontuacao)
                  )

[u'~Unnamed: 0~ExtractedTo~ExtractedFrom~ExtractedBodyText']

In [3]:
print '\n'.join(hillaryDataRDD
                .zipWithIndex()
                .map(lambda (linha, num): '{0}: {1}'.format(num,linha))
                .take(5)
               )

0: 
1: b6 thursday march 3 2011 945 pm h latest how syria is aiding qaddafi and more sid hrc memo syria aiding libya 030311docx hrc memo syria aiding libya 030311docx march 3 2011 for hillary
2: thx
3: 
4: h hrod17clintonemailcom friday march 11 2011 136 pm huma abedin fw h latest how syria is aiding qaddafi and more sid hrc memo syria aiding libya 030311docx pis print

#### ** (4d) Extraindo as palavras **
#### Antes de poder usar nossa função Before we can use the `contaPalavras()`, temos ainda que trabalhar em cima da nossa RDD:
  + #### Precisamos gerar listas de palavras ao invés de listas de sentenças.
  + #### Eliminar linhas vazias.
 
#### As strings em Python tem o método [split()](https://docs.python.org/2/library/string.html#string.split) que faz a separação de uma string por separador. No nosso caso, queremos separar as strings por espaço. 

#### Utilize a função `map()` para gerar um novo RDD como uma lista de palavras.

In [10]:
# EXERCICIO
hillaryPalavrasRDD = hillaryDataRDD.map(lambda x: x.split())
total = hillaryPalavrasRDD.count()
print hillaryPalavrasRDD.take(5)
print total

[[], [u'b6', u'thursday', u'march', u'3', u'2011', u'945', u'pm', u'h', u'latest', u'how', u'syria', u'is', u'aiding', u'qaddafi', u'and', u'more', u'sid', u'hrc', u'memo', u'syria', u'aiding', u'libya', u'030311docx', u'hrc', u'memo', u'syria', u'aiding', u'libya', u'030311docx', u'march', u'3', u'2011', u'for', u'hillary'], [u'thx'], [], [u'h', u'hrod17clintonemailcom', u'friday', u'march', u'11', u'2011', u'136', u'pm', u'huma', u'abedin', u'fw', u'h', u'latest', u'how', u'syria', u'is', u'aiding', u'qaddafi', u'and', u'more', u'sid', u'hrc', u'memo', u'syria', u'aiding', u'libya', u'030311docx', u'pis', u'print']]
7945

#### Conforme deve ter percebido, o uso da função `map()` gera uma lista para cada linha, criando um RDD contendo uma lista de listas.

#### Para resolver esse problema, o Spark possui uma função análoga chamada `flatMap()` que aplica a transformação do `map()`, porém *achatando* o retorno em forma de lista para uma lista unidimensional.

In [4]:
# EXERCICIO
hillaryPalavrasRDD = hillaryDataRDD.flatMap(lambda x: x.split())
total = hillaryPalavrasRDD.count()
print hillaryPalavrasRDD.take(5)
print total

[u'b6', u'thursday', u'march', u'3', u'2011']
595476

#### ** (4f) Contagem de palavras **
#### Agora que nossa RDD contém uma lista de palavras, podemos aplicar nossa função `contaPalavras()`.

#### Aplique a função em nossa RDD e utilize a função `takeOrdered` para imprimir as 15 palavras mais frequentes.

#### `takeOrdered()` pode receber um segundo parâmetro que instrui o Spark em como ordenar os elementos. Ex.:

#### `takeOrdered(15, key=lambda x: -x)`: ordem decrescente dos valores de x

In [12]:
# EXERCICIO
wordCount = (hillaryPalavrasRDD
         .map(lambda x: (x,1))
         .reduceByKey(lambda x,y: x+y)         
        )

top15 = wordCount.takeOrdered(15, key=lambda x: -x[1])

print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15))

the: 31373
to: 17590
and: 14602
of: 13967
a: 11122
in: 10769
that: 6490
for: 6121
is: 5990
on: 5634
i: 4268
with: 4078
you: 3852
it: 3611
as: 3416

In [27]:
from pyspark.sql import Row
from pyspark.sql.types import *

top50 = (wordCount
          .filter(lambda x: len(x[0])>4)
          .takeOrdered(50, key=lambda x: -x[1])
          )

schema = StructType([StructField('word', StringType(), True),
                     StructField('freq', IntegerType(), True)])


top50DF = sqlContext.createDataFrame(top50, schema)
top50DF.registerTempTable('top50')

### Para plotar gráficos no Azure utilizando resultados obtidos do Spark, precisamos armazenar a variável de interesse em uma tabela com o comando:

```python
sqlContext
       .createDataFrame( nome, campos )
       .registerTempTable('nome')
```

### Em seguida, armazenamos uma variável local com a seguinte sequência de comandos:

In [28]:
%%sql -o top50
SELECT * FROM top50

### Para acessar as bibliotecas de plotagem, iniciamos a célula com o comando:

```%%local```

In [ ]:
%%local
%matplotlib inline
import matplotlib.pyplot as plt

points = top50['freq']
labels = top50['word']
plt.figure(figsize=(10,10))
plt.bar(range(len(points)), points)
plt.xticks(range(len(points)), labels, rotation='vertical')
plt.show()

#### O Spark possui duas bibliotecas de Machine Learning, vamos aprender a usar uma delas a **mllib**.

#### Como primeiro passo, vamos transformar nosso texto em uma base de dados TF-IDF, utilizando as funções `HashingTF` e `IDF`.

#### Em seguida, precisamos transformar nossa RDD em uma RDD de tuplas no formato (id, vetorTFIDF).

#### Finalmente, aplicamos o algoritmo LDA para extrair os tópicos da nossa base textual.

In [82]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.linalg import Vectors

dataRDD = (hillaryDataRDD
           .filter(lambda x: len(x))
           )

maxwords = 1000

tfRDD = (HashingTF(numFeatures=maxwords)
         .transform(dataRDD)
         ).cache()

idf = IDF(minDocFreq=10).fit(tfRDD)
tfidfRDD = idf.transform(tfRDD)

trainRDD = (tfidfRDD
            .zipWithIndex()
            .map(lambda x: [x[1], x[0]])
            ).cache()

ldamodel = LDA.train(trainRDD, k=10)

#### Para que os resultados façam sentido, vamos criar um dicionário em que a chave é o índice extraído pelo `HashingTF` e o valor é cada palavra contida em nosso texto.

In [83]:
htf = HashingTF(numFeatures=maxwords)
dictWords = (dataRDD
             .flatMap(lambda x: x.split())
             .distinct()
             .map(lambda x: (htf.indexOf(x),x))
             ).collectAsMap()

#### Vamos imprimir os tópicos para tentar entender o que a Hillary trata em seus e-mails:

In [ ]:
import numpy as np

print("Learned topics (as distributions over vocab of " + str(ldamodel.vocabSize()) + " words):")
topics = ldamodel.topicsMatrix()
for topic in range(10):
    print("Topic " + str(topic) + ":")
    idx = np.argsort([topics[word][topic] for word in range(0,ldamodel.vocabSize())])
    for i in idx[-10:]:
        print dictWords[i],
    print